This script is used to consolidate Mark values for all sites, and join Mark and VIIRS surface reflectance data.

It requires a site location file, "new_site_data_o.csv", an observations file containing dates for all site-seasons, "all_sites_by_day.csv", raw Mark data contained in a folder "hourly_data", and VIIRS surface reflectance values exported from GEE in a folder "batch_export".

Final joined values are exported in the file "all_joined.csv".

In [ ]:

import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Create daily date df

In [ ]:
site_data = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/all_sites_by_day.csv")
site_data_60 = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/new_site_data_o.csv")
site_data_60
unique_sites = site_data_60['Location'].unique()
print(unique_sites)

In [ ]:
out_df = list()
for current_site in unique_sites:
  filtered_df = site_data_60[site_data_60.Location == current_site]
 # print(filtered_df)
  earliest_start = min(pd.to_datetime(filtered_df['new_start']))
  latest_end = max(pd.to_datetime(filtered_df['new_end']))
  #print(earliest_start)
  new_df = pd.DataFrame({ 'date':pd.date_range(earliest_start, latest_end)})
  new_df['Location'] = current_site
  #print(new_df)
  out_df.append(new_df)

out_df = pd.concat(out_df)


join pods

In [ ]:
#pod_d[0].head(20)
joined_s2['s2_round_5min'].median()

TypeError: ignored

In [ ]:
import glob
#prefix = 'ERA5_'
pod_d = list()
for current_site in unique_sites:
  print(current_site)
  s2_data = joined_s2[(joined_s2['Location'] == current_site) & (joined_s2['s2_B8'].notnull())]

  if "choma" in current_site:
    s2_hour_z = 8
  else:
    s2_hour_z = int(s2_data['s2_hour_Z'].median())
 # print(s2_data.head(20))
  print(s2_hour_z)
  if "planet" in current_site:
    current_pod_data = pd.read_csv("/content/gdrive/MyDrive/gee_phd/cal_data/" + current_site + "_CAL.csv")
 #   current_pod_data['']
  #  s2_hour_5min = joined_s2['s2_round_5min'].median().minute
   # print(s2_hour_5min)
  else:
    current_pod_data = pd.read_csv("/content/gdrive/MyDrive/gee_phd/hourly_data/" + current_site + "_HOURLY.csv")


 #print(current_pod_data.head(10))
  current_pod_data['datetime'] = pd.to_datetime(current_pod_data['datetime'])
  current_pod_data['minutes'] = current_pod_data['datetime'].dt.minute
  current_pod_data['hours'] = current_pod_data['datetime'].dt.hour
  current_pod_data['date'] = pd.to_datetime(current_pod_data['date'])
  current_pod_data['datetime_time'] = current_pod_data['datetime'].round('5min').dt.time
  current_pod_data['Location'] = current_site

### filtering
  current_pod_data = current_pod_data[current_pod_data['minutes'] == 0]
  current_pod_data = current_pod_data[current_pod_data['hour_of_day_Z'] == s2_hour_z]


  pod_d.append(current_pod_data)


pod_d = pd.concat(pod_d, join = "inner")
pod_d = pod_d.reset_index()
pod_d = pod_d.drop(['index', 'Unnamed: 0'], axis = 1)



In [ ]:
joined_pod = pd.merge(out_df, pod_d, on=['date', 'Location'], how = 'left')
print(joined_pod.shape)
joined_pod.head(100)
joined_pod.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_pods.csv", encoding = 'utf-8-sig')


(102044, 44)


In [ ]:
joined_pod_dedupe = joined_pod.drop_duplicates(subset=['Location','date'],  keep='first')
print(joined_pod.shape)
print(joined_pod_dedupe.shape)
joined_s2_dedupe = joined_s2.drop_duplicates(subset=['Location','date'],  keep='first')
print(joined_s2.shape)
print(joined_s2_dedupe.shape)
joined_era_dedupe = joined_era.drop_duplicates(subset=['Location','date'],  keep='first')
print(joined_era.shape)
print(joined_era_dedupe.shape)

(102044, 44)
(100871, 44)
(111014, 30)
(100871, 30)
(100871, 54)
(100871, 54)


In [ ]:
joined_pod_dedupe.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_pods_dedupe.csv", encoding = 'utf-8-sig')
joined_s2_dedupe.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_s2_dedupe.csv", encoding = 'utf-8-sig')
joined_era_dedupe.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_era_dedupe.csv", encoding = 'utf-8-sig')


Add VIIRS sr

In [ ]:
gee_files = glob.glob("/content/gdrive/MyDrive/batch_export/" + 'VIIRS_sr_' + "*"  + "*")
gee_files

In [ ]:
import glob
prefix = 'VIIRS_sr_'
sites_d = list()
for current_site in unique_sites:
  print(current_site)
  gee_files = glob.glob("/content/gdrive/MyDrive/batch_export/" + prefix + "*" + current_site + "*")
 # print(gee_files)
  image_data = list()
  for gee in gee_files:
  #  print(gee)
    gee_data = pd.read_csv(gee)
   # print(gee_data.head(5))
    vals = gee_data.iloc[0]
    vals = vals[vals.index != 'system:index']
    vals = vals[vals.index != '.geo']
    dataset = pd.DataFrame({'image_band': vals.index, 'values': vals.values})
  #  print(dataset.head(5))
    dataset['Location'] = current_site
    dataset['image_id'] = (dataset['image_band'].apply(lambda x: ("_".join(x.split("_")[0:3]) )  ))
    dataset['band'] = prefix + dataset['image_band'].apply(lambda x: ("_".join(x.split("_")[3:]) ) )
    dataset['date'] =  pd.to_datetime(dataset['image_band'].apply(lambda x: "".join(x.split("_")[0:3])), format = "%Y%m%d")
    dataset['datetime'] =  pd.to_datetime(dataset['image_band'].apply(lambda x: "_".join(x.split("_")[0:3])), format = "%Y_%m_%d")
    d2 = dataset.pivot(index=['image_id', 'date', 'datetime', 'Location'], columns='band', values='values')
    d2 = d2.reset_index()
    d2 = d2.rename(columns={"datetime": prefix + "datetime", "image_id": prefix + "image_id"})
    sites_d.append(d2)
# #   #joined = pd.merge(out_df, d2, on=['date', 'Location'], how = 'left')

sites_viirs = pd.concat(sites_d)

In [ ]:
sites_viirs.head(400)

In [ ]:
sites_viirs = sites_viirs.reset_index()
for name in sites_viirs.columns:
  print(name)
  if not "date" in name:
    if name == "Location":
      sites_viirs[name] = sites_viirs[name].apply(str)
      continue
    if  "image_id" in name:
      sites_viirs[name] = sites_viirs[name].apply(str)
      continue
    sites_viirs[name] = pd.to_numeric(sites_viirs[name])
sites_viirs = sites_viirs.drop(columns = ['index'])
sites_viirs = sites_viirs.rename_axis(None, axis=1)

# sites_d.columns
print(sites_viirs.shape)
# # sites_d.index.name = None
sites_viirs.head(40)
# # sites_d['band']

In [ ]:
joined_viirs = pd.merge(out_df, sites_viirs, on=['date', 'Location'], how = 'left')
joined_viirs.head(40)
joined_viirs.shape
joined_viirs.columns

Index(['date', 'Location', 'VIIRS_sr_image_id', 'VIIRS_sr_datetime',
       'VIIRS_sr_I1', 'VIIRS_sr_I2', 'VIIRS_sr_I3', 'VIIRS_sr_M1',
       'VIIRS_sr_M10', 'VIIRS_sr_M11', 'VIIRS_sr_M2', 'VIIRS_sr_M3',
       'VIIRS_sr_M4', 'VIIRS_sr_M5', 'VIIRS_sr_M7', 'VIIRS_sr_M8',
       'VIIRS_sr_QF1', 'VIIRS_sr_QF2', 'VIIRS_sr_QF3', 'VIIRS_sr_QF4',
       'VIIRS_sr_QF5', 'VIIRS_sr_QF6', 'VIIRS_sr_QF7',
       'VIIRS_sr_SensorAzimuth', 'VIIRS_sr_SensorZenith',
       'VIIRS_sr_SolarAzimuth', 'VIIRS_sr_SolarZenith', 'VIIRS_sr_iobs_res',
       'VIIRS_sr_num_observations_1km', 'VIIRS_sr_num_observations_500m',
       'VIIRS_sr_obscov_1km', 'VIIRS_sr_obscov_500m', 'VIIRS_sr_orbit_pnt'],
      dtype='object')

In [ ]:
joined_viirs.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_viirs.csv", encoding = 'utf-8-sig')

In [ ]:
joined_viirs_dedupe = joined_viirs.drop_duplicates(subset=['Location','date'],  keep='first')
print(joined_viirs.shape)
print(joined_viirs_dedupe.shape)

(100871, 33)
(100871, 33)


In [ ]:
joined_viirs_dedupe.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_viirs_dedupe.csv", encoding = 'utf-8-sig')

RELOAD DATA AND ADD BANDS (BEFORE JOINING)

In [ ]:
def add_vi_dates(df, prefix, nir, red, green, datetime, date, datetimeConvert):
  df[prefix + "ndvi"] = (df[nir] - df[red])/(df[nir] + df[red])
  df[prefix + "savi"] = 1.5 *(df[nir] - df[red])/(df[nir] + df[red]+ 0.5)
  df[prefix + "evi2"] = 2.5 * (df[nir] - df[red])/(df[nir] + (2.4 *df[red]) + 1.0)
  df[prefix + "gcvi"] = (df[nir]/df[green]) - 1.0
  if datetimeConvert:
    df['datetime'] = pd.to_datetime(df[datetime].map(str), format = "%Y-%m-%d %H:%M:%S")
  df['date'] = pd.to_datetime(df[date], format = "%Y-%m-%d")
  return(df)

def add_s1_vi_dates(df, prefix, vv, vh, datetime, date, datetimeConvert):
  df[prefix + "vv_vh"] = (df[vv] - df[vh])
  df[prefix + "vh_vv"] = (df[vh] - df[vv])
 # df[prefix + "rvi"] = 2.5 * (df[nir] - df[red])/(df[nir] + (2.4 *df[red]) + 1.0)
  if datetimeConvert:
    df['datetime'] = pd.to_datetime(df[datetime].map(str), format = "%Y-%m-%d %H:%M:%S")
  df['date'] = pd.to_datetime(df[date], format = "%Y-%m-%d")
  return(df)



def add_lag_variables(df, cols, lags, fun = 'mean'):
  for col in cols:
    for lag in lags:
      if fun == 'mean':
        df[col + "_lag_" + str(lag)] = df.groupby('Location')[col].transform(lambda x: x.rolling(lag, 1).mean())
      if fun == 'sum':
        df[col + "_lag_" + str(lag)] = df.groupby('Location')[col].transform(lambda x: x.rolling(lag, 1).sum())
      if fun == 'median':
        df[col + "_lag_" + str(lag)] = df.groupby('Location')[col].transform(lambda x: x.rolling(lag, 1).median())
  return(df)




In [ ]:
pod_dedupe = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_pods_dedupe.csv")

In [ ]:
pod_dedupe.head(100)

In [ ]:
import numpy as np
pod_dedupe = add_vi_dates(pod_dedupe, "", "b6_sr", "b4_sr","b2_sr", "datetime", "date", False)
pod_dedupe['pod_quality'] = (pod_dedupe['ndvi'] > 0)
pod_dedupe.loc[pod_dedupe['pod_quality'] == False, 'gcvi'] = np.NaN
pod_dedupe.loc[pod_dedupe['pod_quality'] == False, 'savi'] = np.NaN
pod_dedupe.loc[pod_dedupe['pod_quality'] == False, 'evi2'] = np.NaN
pod_dedupe.loc[pod_dedupe['pod_quality'] == False, 'ndvi'] = np.NaN

pod_dedupe[pod_dedupe['ndvi'] < 0].head(1000)
#pod_dedupe.dtypes

,Unnamed: 0,date,Location,lat,b7uw,b5uw,swuw,tbelow,lwdw,b2dw,b3uw,tabove,rh,b5dw,b1dw,b2uw,long,b6uw,b4uw,b7dw,b3dw,lwuw,swdw,tair,b6dw,b1uw,p,b4dw,device,datetime,hour_of_day_Z,b1_sr,b2_sr,b3_sr,b4_sr,b5_sr,b6_sr,b7_sr,lw_sr,sw_sr,site_id,group_id,minutes,hours,datetime_time,ndvi,savi,evi2,gcvi,pod_quality


In [ ]:
print(pod_dedupe.shape)
print(pod_dedupe[pod_dedupe['ndvi'].notnull()].shape)
print(pod_dedupe[pod_dedupe['ndvi']>1].shape)

(100871, 50)
(37421, 50)
(0, 50)


In [ ]:
s2_dedupe = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_s2_dedupe.csv")
s2_dedupe = add_vi_dates(s2_dedupe, "s2_", "s2_B8", "s2_B4", "s2_B3", "s2_datetime", "date", True )
s2_dedupe.dtypes

In [ ]:
import numpy as np

s2_dedupe['quality'] = (s2_dedupe.s2_QA60.notnull()) & (s2_dedupe.s2_QA60 == 0) & (s2_dedupe.s2_SCL.isin([4,5]))
cols = s2_dedupe.columns
s2_cols = [col for col in cols if 's2_' in col]

for col in s2_cols:
  s2_dedupe.loc[s2_dedupe['quality'] == False, col] = np.NaN

s2_dedupe.head(200)
print(s2_dedupe[(s2_dedupe.s2_QA60.notnull()) & (s2_dedupe.s2_QA60 == 0) & (s2_dedupe.s2_SCL.isin([4,5]))].shape)
print(s2_dedupe.shape)

(10238, 37)
(100871, 37)


In [ ]:
s2_dedupe = add_lag_variables(s2_dedupe, ['s2_ndvi', 's2_gcvi', 's2_savi', 's2_evi2'], [30, 60])
s2_dedupe.head(500)

In [ ]:
s2_dedupe['s2_most_recent_valid_ndvi'] = s2_dedupe['s2_ndvi'].fillna(method = 'ffill')
s2_dedupe['s2_most_recent_valid_gcvi'] = s2_dedupe['s2_gcvi'].fillna(method = 'ffill')
s2_dedupe['s2_most_recent_valid_savi'] = s2_dedupe['s2_savi'].fillna(method = 'ffill')
s2_dedupe['s2_most_recent_valid_evi2'] = s2_dedupe['s2_evi2'].fillna(method = 'ffill')

s2_dedupe.loc[s2_dedupe['quality'] == True, 's2_days_since_last'] = 0
s2_dedupe.loc[s2_dedupe['quality'] == False, 's2_days_since_last'] = np.NaN

s2_dedupe['s2_days_since_last'] = s2_dedupe.s2_days_since_last.ffill()+ s2_dedupe.groupby(s2_dedupe.s2_days_since_last.notnull().cumsum()).cumcount()/1

s2_dedupe.head(500)

In [ ]:
s1_dedupe = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_s1_dedupe.csv")
s1_dedupe.dtypes
s1_dedupe = add_s1_vi_dates(s1_dedupe, "s1_", "s1_VV", "s1_VH", "s1_datetime", "date", True)
s1_dedupe = add_lag_variables(s1_dedupe, ['s1_VV', 's1_VH', 's1_vh_vv', 's1_vv_vh'], [30, 60], fun = 'median')


In [ ]:
s1_dedupe.head(500)

VIIRS

In [ ]:
viirs_dedupe = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_viirs_dedupe.csv")
viirs_dedupe = add_vi_dates(viirs_dedupe, "VIIRS_sr_", "VIIRS_sr_M7", "VIIRS_sr_M5", "VIIRS_sr_M4", "VIIRS_sr_datetime", "date", True )
# viirs_dedupe = add_lag_variables(viirs_dedupe, ['ERA5_temperature_2m',
#                                             'ERA5_leaf_area_index_high_vegetation',
#                                             'ERA5_leaf_area_index_low_vegetation',
#                                             'ERA5_surface_solar_radiation_downwards_sum',
#                                             'ERA5_total_precipitation_sum'], [30, 60])
viirs_dedupe.head(500)
viirs_dedupe.shape

(100871, 39)

In [ ]:
viirs_dedupe = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/joined_viirs_dedupe.csv")
viirs_dedupe = add_vi_dates(viirs_dedupe, "VIIRS_sr_", "VIIRS_sr_M7", "VIIRS_sr_M5", "VIIRS_sr_M4", "VIIRS_sr_datetime", "date", True )
# viirs_dedupe = add_lag_variables(viirs_dedupe, ['ERA5_temperature_2m',
#                                             'ERA5_leaf_area_index_high_vegetation',
#                                             'ERA5_leaf_area_index_low_vegetation',
#                                             'ERA5_surface_solar_radiation_downwards_sum',
#                                             'ERA5_total_precipitation_sum'], [30, 60])
viirs_dedupe.head(500)
viirs_dedupe.shape

In [ ]:
print(pod_dedupe.shape)
all_joined = pod_dedupe.merge(s2_dedupe, on=['date', 'Location'], how = 'left').merge(s1_dedupe, on=['date', 'Location'], how = 'left').merge(era_dedupe, on=['date', 'Location'], how = 'left').merge(viirs_dedupe, on = ['date', 'Location'], how = 'left').merge(MOD09GA_dedupe, on = ['date', 'Location'], how = 'left').merge(MYD09GA_dedupe, on = ['date', 'Location'], how = 'left')
print(all_joined.shape)
all_joined.head(500)
all_joined.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/all_joined.csv")

(100871, 50)
(100871, 276)


Filtering

In [ ]:
all_filt = list()

for current_site in unique_sites:

  print(current_site)
  df_filt = all_joined[all_joined['Location'] == current_site]
 # df_filt = df_filt.reset_index()
#  print(df_filt.shape)
  df_filt = df_filt.reset_index()
  first_s2 = df_filt[df_filt.quality == True].first_valid_index()
  if first_s2 == None:
    print('no valid obs')
    continue
#  df_filt = df_filt.iloc[first_s2:]
  print(first_s2)
  print(first_s2 == None)
  all_filt.append(df_filt)
#df_filt.head(10)
all_filt = pd.concat(all_filt)
print(all_filt.shape)
all_filt.head(500)

#df_filt.head(20)

## filter out Whittier 2020 midslope
## filter out all observations before first valid S2 obs.

In [ ]:
#all_joined[all_joined['Location'] == unique_sites[1]].iloc[5:]

In [ ]:
all_filt.shape

(95891, 277)

In [ ]:
#all_filt[(all_filt.ERA5_temperature_2m.isnull()) & (all_filt.ndvi.notnull())].head(300)
all_filt = all_filt[all_filt.Location != 'whittier_midslope_2020' ]
print(all_filt.shape)

all_filt_valid_pod = all_filt[all_filt.ndvi.notnull()]
print(all_filt_valid_pod.shape)


(95618, 277)
(33481, 277)


In [ ]:
all_filt.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/all_filt.csv")
all_filt_valid_pod.to_csv("/content/gdrive/MyDrive/gee_phd/site_locations/all_filt_valid_pod.csv")